# Pipeline de Ensemble por Modelos (SA → MA)

Este notebook detalha, passo a passo, o processo de ensemble em dois níveis:

- Single-Architecture (SA): ensemble por modelo, nos níveis Tile → Image → Patient.
- Multi-Architecture (MA): ensemble entre diferentes modelos, nos níveis Image → Tile → Patient.

Ao final, os artefatos (CSVs e métricas) são organizados em 
`outputs/tables` (tabelas geradas) e `outputs/results` (artefatos para consulta/apresentação).


## 1. Configuração Geral
Defina: modelos a incluir, tipo de ensemble e métrica de peso (quando weighted).
Os dados de entrada dos folds por arquitetura estão em `datas/summary_results_<modelo>/`.

In [1]:
from modules.metrics import generate_ma_metrics_from_tables, generate_sa_metrics_from_tables
from modules.utils import log, discover_models_and_paths
from modules.utils import resolve_paths_outputs
import modules.flags as flags
import os, json


In [2]:

# Configurar flags (plots opcionais para soft_voting)
flags.GENERATE_LEVELS = ['tile', 'image', 'patient']
if 'soft_voting' not in flags.GENERATE_PLOTS_FOR:
    flags.GENERATE_PLOTS_FOR.append('soft_voting')

MODELS = ['EFFNet', 'GGNet', 'MOBNet']
ENSEMBLE_TYPE = 'soft_voting'  # 'hard_voting' | 'soft_voting' | 'weighted'
WEIGHT_METRIC = 'f1_macro'     # usado apenas com 'weighted'
TABLES_DIR, _ = resolve_paths_outputs()
OUTPUT_FILENAME = "tasks.json"


## 2. Descoberta dos CSVs de Folds (datas/)
Mostra quantos CSVs foram encontrados por arquitetura e um exemplo de arquivo.

In [3]:

print(f"--- Iniciando Preparação de Tarefas ---")

try:
    # 2. DESCUBRA OS MODELOS E SEUS CAMINHOS
    models_and_paths = discover_models_and_paths(models_to_find=MODELS)

    if not models_and_paths:
        print("[INFO] Nenhum modelo válido foi encontrado. Nenhum arquivo de tarefas gerado.")
        exit()

    # 3. PREPARE A LISTA DE CONFIGURAÇÕES (O CONTEÚDO DO FUTURO JSON)
    configs_to_save = []
    for model_name, csv_path_list in models_and_paths.items():
        if not csv_path_list:
            print(f"[AVISO] Modelo '{model_name}' não possui CSVs e será ignorado.")
            continue

        config = {
            'model_name': model_name,
            'ensemble_type': ENSEMBLE_TYPE,
            'weight_metric': WEIGHT_METRIC,
            'csv_paths': csv_path_list,
        }
        configs_to_save.append(config)
    
    # 4. SALVE AS CONFIGURAÇÕES EM UM ARQUIVO JSON
    with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
        json.dump(configs_to_save, f, indent=4)

    print(f"\n[SUCESSO] Arquivo '{OUTPUT_FILENAME}' gerado com {len(configs_to_save)} tarefas.")
    print("Agora você pode mover este arquivo e rodar o script 'execute_tasks.py'.")

except FileNotFoundError as e:
    print(f"\n[ERRO CRÍTICO] {e}")
except Exception as e:
    print(f"\n[ERRO INESPERADO] Ocorreu um erro durante a preparação: {e}")


--- Iniciando Preparação de Tarefas ---
[INFO] Buscando modelos especificados: ['EFFNet', 'GGNet', 'MOBNet']

--- Relatório de Descoberta ---
EFFNet: 10 CSVs encontrados
------------------------------------------------------------
GGNet: 10 CSVs encontrados
------------------------------------------------------------
MOBNet: 10 CSVs encontrados
------------------------------------------------------------
[INFO] Modelos configurados para execução: ['EFFNet', 'GGNet', 'MOBNet']

[SUCESSO] Arquivo 'tasks.json' gerado com 3 tarefas.
Agora você pode mover este arquivo e rodar o script 'execute_tasks.py'.


## 3. Ensemble por Modelo (SA): Tile → Image → Patient
Nesta etapa consolidamos os folds de um mesmo modelo em três níveis.
Os resultados são salvos em `outputs/tables/<Modelo>/Ensemble_<level>_<tipo>/`.
Depois, exportamos artefatos para `outputs/results/SA/<Modelo>/<tipo>/<Level>/`.

In [4]:
from modules.runner_sa import run_sa_for_models_parallel
# Execução do pipeline SA

INPUT_FILENAME = "tasks.json"
if not os.path.exists(INPUT_FILENAME):
    raise FileNotFoundError(f"Arquivo de plano de execução '{INPUT_FILENAME}' não encontrado!")

with open(INPUT_FILENAME, 'r', encoding='utf-8') as f:
    lista_de_configs = json.load(f)

log("[SA] Iniciando execução com paralelização e cache interno (tile → image → patient)...")
# Sumário dos artefatos gerados
# -----------------------------------------
run_sa_for_models_parallel(TABLES_DIR, lista_de_configs, max_workers=4, use_threads=True)


[13:53:05] [SA] Iniciando execução com paralelização e cache interno (tile → image → patient)...
[13:53:05] [SA] Iniciando execução do pipeline SA para 3 modelos (tile → image → patient)...
[13:53:05] [INICIANDO] Processamento do modelo: EFFNet
[13:53:05] [INICIANDO] Processamento do modelo: GGNet
[13:53:05] [SA] EFFNet → tile
[13:53:05] [INICIANDO] Processamento do modelo: MOBNet
[13:53:05] [SA] GGNet → tile
[13:53:05] [SA] MOBNet → tile
[14:02:09] [SA] EFFNet → image
[14:02:19] [SA] MOBNet → image
[14:02:32] [SA] GGNet → image
[14:04:19] [SA] EFFNet → patient
[14:04:36] [SA] MOBNet → patient
[14:04:52] [SA] GGNet → patient
[14:06:28] [AVISO] Falha ao exportar SA para EFFNet: name 'export_sa_to_results' is not defined
[14:06:28] [CONCLUÍDO] Processamento do modelo: EFFNet
[14:06:41] [AVISO] Falha ao exportar SA para MOBNet: name 'export_sa_to_results' is not defined
[14:06:41] [CONCLUÍDO] Processamento do modelo: MOBNet
[14:06:50] [AVISO] Falha ao exportar SA para GGNet: name 'export_

#### **[SA]  Gerar Métricas**

In [ ]:
generate_sa_metrics_from_tables()

## 4. Ensemble Entre Modelos (MA)
Combina as saídas SA dos modelos selecionados.
Os resultados são salvos em `outputs/tables/Ensemble_Between_Models/...` 
e exportados para `outputs/results/MA/<tipo>/<Level>/`.

In [6]:
import os
from modules.runner_ma import run_ma_for_models_parallel
from modules.utils import log 


# =======================================================================
# 2. EXECUTE O PIPELINE
# =======================================================================

log(f'\n[MA] Rodando ensemble entre modelos para os tipos: {ENSEMBLE_TYPE} ...')

run_ma_for_models_parallel(
    tables_dir=TABLES_DIR,
    models_to_include=MODELS,
    ensemble_type=ENSEMBLE_TYPE, 
    weight_metric=WEIGHT_METRIC
)

[14:06:50] 
[MA] Rodando ensemble entre modelos para os tipos: soft_voting ...
[14:06:50] [MA] Iniciando pipeline de Ensemble Entre Modelos para: ['EFFNet', 'GGNet', 'MOBNet']
[14:06:50] [MA] Tipo de Ensemble a ser executado: 'soft_voting'
[14:06:50] [MA] Resultados serão salvos em: c:\Users\Fernando Alves\Desktop\Nova pasta\ensamble_pipeline_MA_SA_models\outputs\tables\Ensemble_Between_Models
[14:06:50] [MA INICIANDO] Executando ensemble 'soft_voting' para o nível 'tile'...
[14:06:50] [MA INICIANDO] Executando ensemble 'soft_voting' para o nível 'image'...
[14:06:50] [MA INICIANDO] Executando ensemble 'soft_voting' para o nível 'patient'...
[14:06:50] [MA SUCESSO] Nível 'image' (soft_voting) concluído. CSV salvo em: c:\Users\Fernando Alves\Desktop\Nova pasta\ensamble_pipeline_MA_SA_models\outputs\tables\Ensemble_Between_Models\ImageLevel_Ensemble_Models_soft_voting\ensemble_between_models_per_image_soft_voting.csv
[14:06:50] [MA SUCESSO] Nível 'patient' (soft_voting) concluído. CSV sa

#### **[MA]  Gerar Métricas**

In [8]:
generate_ma_metrics_from_tables()


[14:37:58] [METRICS] Salvo: c:\Users\Fernando Alves\Desktop\Nova pasta\ensamble_pipeline_MA_SA_models\outputs\results\MA\tile\metrics_tile_level_soft_voting.json
[14:38:03] [METRICS] Salvo: c:\Users\Fernando Alves\Desktop\Nova pasta\ensamble_pipeline_MA_SA_models\outputs\results\MA\image\metrics_image_level_soft_voting.json
[14:38:04] [METRICS] Salvo: c:\Users\Fernando Alves\Desktop\Nova pasta\ensamble_pipeline_MA_SA_models\outputs\results\MA\patient\metrics_patient_level_soft_voting.json
